In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import requests
from datetime import datetime
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors as mcolors
import matplotlib.patheffects as PathEffects
import metpy as mp
import metpy.calc as mpcalc
from metpy.units import units
import numpy as np
from scipy.ndimage import gaussian_filter
from scipy import spatial
import xarray as xr
import sys
import netCDF4 as nc

In [ ]:
api_auth = '/home/tyler/.api_request'
url = "https://forecasting.energy.arizona.edu/erebos/series/adjghi"

In [ ]:
startlon = -110.95534
startlat = 32.22969

In [ ]:
latz = []
lonz = []

In [ ]:
for i in range(0,7):
    dalats = startlat + (0.009 * i) 
    dalons = startlon + (0.009 * i)
    latz.append(dalats)
    lonz.append(dalons)

In [ ]:
print("LATS:", latz)
print("LONS:",lonz)

In [ ]:
ghi_test = []

In [ ]:
for i in range(len(latz)):
    for j in range(len(lonz)):
        args = {'run_date':'2021-07-11', 
                'lon':'{}'.format(lonz[j]),
                'lat':'{}'.format(latz[i]),
                'precipitable_water':'1',
                'aod700':'0.05'}
        with open(api_auth) as f:
            auth_text = f.read()
        auth_tuple = tuple(auth_text.split('\n'))[:2]
        x = requests.get(url, params=args, auth=auth_tuple)
        df = pd.DataFrame(x.json())
        #df.reindex(pd.to_datetime(df.index))
        if i == 0:
            date = pd.to_datetime(df.index)
        
        ghi = df['results'].squeeze().values
        ghi_test.append(ghi[13])

In [ ]:
ghi1 = np.asarray(ghi_test)
latz = np.asarray(latz)
lonz = np.asarray(lonz)

In [ ]:
ghi1 = ghi1.reshape(7,7)

In [ ]:
filename = 'test13.nc'

In [ ]:
ds = nc.Dataset(filename, 'w', format='NETCDF4')

In [ ]:
time = ds.createDimension('time', None)
lat = ds.createDimension('lat', 7)
lon = ds.createDimension('lon', 7)

In [ ]:
times = ds.createVariable('time', 'f4', ('time',))
lats = ds.createVariable('lat', 'f4', ('lat'))
lons = ds.createVariable('lon', 'f4', ('lon'))
ghi = ds.createVariable('ghi', 'f4', ('time', 'lat', 'lon',))
ghi.units = 'Unknown'

In [ ]:
lats[:] = latz
lons[:] = lonz

In [ ]:
print(latz,lonz)

In [ ]:
ghi[0, :, :] = ghi1

In [ ]:
ds.close()